In [267]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Strategy
from backtesting import Backtest
from sklearn.model_selection import ParameterGrid

In [268]:
def getYFinanceData(ticker, period, interval):
  dataF = yf.download(ticker, period=period, interval=interval)

  dataF.iloc[:,:]
  #dataF.Open.iloc

  df = pd.DataFrame(dataF)

  # use df index, convert DateTime to  a column instead of index
  df.reset_index(inplace=True)

  # delete Adj Close
  df = df.drop(['Adj Close'], axis=1)

  # rename Datetime to "Gmt time"
  df = df.rename(columns={'Datetime':'Gmt time'})
  
  # rename Date to "Gmt time"
  df = df.rename(columns={'Date':'Gmt time'})

  df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
  df.set_index("Gmt time", inplace=True)
  df=df[df.High!=df.Low]

  return df

In [269]:
def calculateTechnicalAnalysis(df):
  df["MACD"]=ta.macd(df.Close)['MACD_12_26_9']
  df["MACD_HIST"]=ta.macd(df.Close)['MACDh_12_26_9']
  df["MACD_SIGNAL"]=ta.macd(df.Close)['MACDs_12_26_9']
  df['RSI']=ta.rsi(df.Close, length=16)
  return df

In [270]:
def getDataPair(ticker):
  df = getYFinanceData(ticker, "730d", "60m")
  df1d = getYFinanceData(ticker, "730d", "1d")
  
  df = calculateTechnicalAnalysis(df)
  df1d = calculateTechnicalAnalysis(df1d)
  
  # create the column called Date. its value is the index without the time
  df['Date'] = df.index.date
  df1d['Date'] = df1d.index.date
  
  # Ensure 'Date' in df and index in df1d are in the same datetime format
  df['Date'] = pd.to_datetime(df['Date'])
  df1d.index = pd.to_datetime(df1d.index)

  # Use a try/except block to handle missing dates
  def get_macd(row):
      try:
          return df1d.loc[row['Date']]['MACD']
      except KeyError:
          return np.nan
      
      
  def get_macd_hist(row):
      try:
          return df1d.loc[row['Date']]['MACD_HIST']
      except KeyError:
          return np.nan
      
  def get_macd_signal(row):
      try:
          return df1d.loc[row['Date']]['MACD_SIGNAL']
      except KeyError:
          return np.nan
      
  def get_adx(row):
      try:
          return df1d.loc[row['Date']]['ADX']
      except KeyError:
          return np.nan
      


  df['MACD_1d'] = df.apply(get_macd, axis=1)
  df['MACD_HIST_1d'] = df.apply(get_macd_hist, axis=1)
  df['MACD_SIGNAL_1d'] = df.apply(get_macd_signal, axis=1)
  df['ADX_1d'] = df.apply(get_adx, axis=1)
  return df

MACD Signal

In [271]:
def getMACDSignals(df):
  backcandles = 15
  # calcualte macd signal
  macd_signal = [0]*len(df)
  for row in range(1, len(df)):
      if(
          True
          and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
      ):
          macd_signal[row]=1
      elif  (
          True
          and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
      ):
          macd_signal[row]=-1

  df['MACDSignal'] = macd_signal
  return df

RSI Signal

In [272]:
def getRSISignals(df):
  backcandles = 14

  # calcualte macd signal
  rsi_signal = [0]*len(df)
  df.RSI_14MA = df.RSI
  for row in range(1, len(df)):
      if row < backcandles:
          continue

      if(
          True
          and df.RSI[row] > 50
          or (df.RSI[row]<25 and df.RSI[row-1]>25)

      ):
          rsi_signal[row]=1
      elif  (
          True
          and df.RSI[row] < 55
          or (df.RSI[row]>75 and df.RSI[row-1]<75)
      ):
          rsi_signal[row]=-1

  df['RSI_Signal'] = rsi_signal
  return df

Total Signal

In [273]:
def TotalSignal(df, l):
  if (
      df.MACDSignal[l]==1
      and df.RSI_Signal[l]==1
  ):
          return 2
  if (
      df.MACDSignal[l]==-1
      and df.RSI_Signal[l]==-1
  ):
          return 1
  return 0
      
def assignTotalSignal(df):
  backcandles= 14
  TotSignal = [0]*len(df)
  for row in range(backcandles, len(df)): #careful backcandles used previous cell
      TotSignal[row] = TotalSignal(df, row)
  df['TotalSignal'] = TotSignal
  return df

In [274]:
yfinance_dict = {}

def backtest(ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
    df = None
    if (ticker in yfinance_dict):
        df = yfinance_dict[ticker]
    else :
        df = getDataPair(ticker)
        df = getMACDSignals(df)
        df = getRSISignals(df)
        df = assignTotalSignal(df)
        yfinance_dict[ticker] = df
    
    dfpl = df.copy()
    import pandas_ta as ta
    dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=atr_length)
    #help(ta.atr)
    def SIGNAL():
        return dfpl.TotalSignal

    class MyStrat(Strategy):
        initsize = init_size
        mysize = initsize
        latestEntry = 0
        lastHigh = 0
        
        def init(self):
            super().init()
            self.signal1 = self.I(SIGNAL)

        def next(self):
            super().next()
            slatr = slatr_coef*self.data.ATR[-1]
            TPSLRatio = tp_sl_ratio
            
            # if the current equity is higher than lastHigh, then update lastHigh
            if (self.equity > self.lastHigh):
                self.lastHigh = self.equity
                
            # if self.position.pl is down for more than 2 percent of the height, close all positions
            if (self.position.pl < -1 * self.lastHigh * 0.02):
                self.position.close()
                # print("Close position due to loss")
                # print("------")
            
            # # close the position if the amount lost exceeds 3% of the account balance
            if (self.position.pl < -1 * self.equity * 0.02):
                self.position.close()
                # print("Close position due to loss")
                # print("------")

            if len(self.trades)>0:
                for trade in self.trades:
                    if trade.is_long and self.data.RSI[-1]>=90:
                        trade.close()
                    elif trade.is_short and self.data.RSI[-1]<=10:
                        trade.close()
                    
                    # update the stop loss to entry if the pl is greater than the stop loss amount
                    # if (trade.pl > self.equity * 0.01):
                    #     trade.sl = trade.entry_price
                    #     print("Update stop loss to entry")
                    #     print("------")
                    
                    # close the position if the amount lost exceeds 3% of the account balance
                    # if (trade.pl < -1 * self.equity * 0.01):
                    #     trade.close()
                    #     print("Close position due to loss")
                    #     print("------")
                    
            # # # if the pl is greater than the stop loss amount, then move the stop loss to entry
            # if (self.position.pl > self.latestEntry * 0.02):
            #     # adjust the stop loss to entry
            #     self.position.sl = self.latestEntry
            #     # # for each opened position, update the stop loss to the entry
            #     # for trade in self.trades:
            #     #     trade.sl = self.latestEntry
                    
                    
            
            # print("Datetime: ", self.data.index[-1])
            if self.signal1==2 and len(self.trades) < 3:
                sl1 = self.data.Close[-1] - slatr
                tp1 = self.data.Close[-1] + slatr*TPSLRatio
                self.buy(sl=sl1, tp=tp1, size=self.mysize)
                self.latestEntry = self.data.Close[-1]
                # print("Buy", self.data.Close[-1])
                # print("Stop loss: ", sl1)
                # print("Take profit: ", tp1)
                # print("------")
            
            elif self.signal1==1 and len(self.trades) < 1:         
                sl1 = self.data.Close[-1] + slatr
                tp1 = self.data.Close[-1] - slatr*TPSLRatio
                self.sell(sl=sl1, tp=tp1, size=self.mysize)
                self.latestEntry = self.data.Close[-1]
                # print("Sell", self.data.Close[-1])
                # print("Stop loss: ", sl1)
                # print("Take profit: ", tp1)
                # print("------")

            # else:
            #     print("No signal")

    bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/100, commission=0)
    stat = bt.run()
    return stat, bt

---

## Optimize Draw Down

In [235]:
# use gridsearchcv to find the best parameters

# turn off printing
import sys
sys.stdout = open('output.txt', 'w')

# Define your custom estimator and scoring method
class MyCustomEstimator:
    def __init__(self, ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
        self.ticker = ticker
        self.atr_length = atr_length
        self.init_size = init_size
        self.tp_sl_ratio = tp_sl_ratio
        self.slatr_coef = slatr_coef

    def score(self):
        # Your scoring logic here
        stat, bt = backtest(self.ticker, atr_length=self.atr_length, init_size=self.init_size , tp_sl_ratio=self.tp_sl_ratio, slatr_coef=self.slatr_coef)
        return stat["Max. Drawdown [%]"]

# Create a grid of hyperparameters
param_grid = {
    'ticker': ["^NDX", "^SPX", "AAPL"],
    'atr_length': [1, 4, 8, 12, 16],
    'init_size': [0.02, 0.03],
    'tp_sl_ratio': [1.5, 2],
    'slatr_coef': [4, 9, 12],
}

parameters_df= []

# Iterate over all combinations
for params in ParameterGrid(param_grid):
    estimator = MyCustomEstimator(**params)
    score = estimator.score()
    # Compare scores and find the best combination
    parameters_df.append([params, score])

parameters_df

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3184709326.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3184709326.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3003789530.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.RSI_14MA = d

[[{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  -31.18394032445091],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  -30.44739452018299],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  -34.16920308170492],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 2},
  -36.17004402085594],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  -33.490283424926304],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 2},
  -37.506748253583474],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  -47.427741307720225],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_

In [236]:
# find thee param tha resulted in the highest score
# max(parameters_df, key=lambda x: x[1])

ticker_groups = {}
for item in parameters_df:
    ticker = item[0]['ticker']
    if ticker not in ticker_groups:
        ticker_groups[ticker] = []
    ticker_groups[ticker].append(item)

# Find the item with the highest value in index 1 for each ticker
result = {}
for ticker, items in ticker_groups.items():
    max_item = max(items, key=lambda x: x[1])
    result[ticker] = max_item

result

{'^NDX': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  -30.44739452018299],
 '^SPX': [{'atr_length': 8,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  -26.469594857964495],
 'AAPL': [{'atr_length': 4,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  -32.35027169284598]}

In [300]:
stat, bt = backtest("^SPX", atr_length=8, init_size=0.03 , tp_sl_ratio=1.5, slatr_coef=4)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/69321050.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2021-06-07 09:30:00
End                       2024-03-25 13:30:00
Duration                   1022 days 04:00:00
Exposure Time [%]                   61.740191
Equity Final [$]                429122.867758
Equity Peak [$]                  436452.46102
Return [%]                         329.122868
Buy & Hold Return [%]               23.857585
Return (Ann.) [%]                   68.311696
Volatility (Ann.) [%]               84.231752
Sharpe Ratio                         0.810997
Sortino Ratio                        2.461927
Calmar Ratio                         2.180441
Max. Drawdown [%]                  -31.329297
Avg. Drawdown [%]                   -4.151761
Max. Drawdown Duration      316 days 21:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                  158
Win Rate [%]                        36.708861
Best Trade [%]                       7.058676
Worst Trade [%]                     -3.749627
Avg. Trade [%]                    

In [304]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=2, slatr_coef=4)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/1630965305.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2021-04-30 09:30:00
End                       2024-03-25 14:30:00
Duration                   1060 days 05:00:00
Exposure Time [%]                   69.779788
Equity Final [$]                535038.649414
Equity Peak [$]                 692238.485352
Return [%]                         435.038649
Buy & Hold Return [%]               31.297349
Return (Ann.) [%]                    78.56135
Volatility (Ann.) [%]                90.42001
Sharpe Ratio                         0.868849
Sortino Ratio                        2.772082
Calmar Ratio                         2.580232
Max. Drawdown [%]                  -30.447395
Avg. Drawdown [%]                    -2.86957
Max. Drawdown Duration      266 days 02:00:00
Avg. Drawdown Duration        8 days 12:00:00
# Trades                                  143
Win Rate [%]                        37.062937
Best Trade [%]                      13.674375
Worst Trade [%]                     -4.340308
Avg. Trade [%]                    

## Optimize Win Rate

In [288]:
# use gridsearchcv to find the best parameters

# turn off printing
import sys
sys.stdout = open('output.txt', 'w')

# Define your custom estimator and scoring method
class MyCustomEstimator:
    def __init__(self, ticker, atr_length, init_size, tp_sl_ratio, slatr_coef):
        self.ticker = ticker
        self.atr_length = atr_length
        self.init_size = init_size
        self.tp_sl_ratio = tp_sl_ratio
        self.slatr_coef = slatr_coef

    def score(self):
        # Your scoring logic here
        stat, bt = backtest(self.ticker, atr_length=self.atr_length, init_size=self.init_size , tp_sl_ratio=self.tp_sl_ratio, slatr_coef=self.slatr_coef)
        return stat["Win Rate [%]"]

# Create a grid of hyperparameters
param_grid = {
    'ticker': ["^NDX", "^SPX", "AAPL"],
    'atr_length': [1, 4, 8, 12, 16],
    'init_size': [0.02, 0.03],
    'tp_sl_ratio': [1.5, 2],
    'slatr_coef': [4, 9, 12],
}

parameters_df= []

# Iterate over all combinations
for params in ParameterGrid(param_grid):
    estimator = MyCustomEstimator(**params)
    score = estimator.score()
    # Compare scores and find the best combination
    parameters_df.append([params, score])

parameters_df

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3184709326.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] < 0 and df.MACD_HIST[row] > 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3184709326.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  and df.MACD_HIST[row-1] > 0 and df.MACD_HIST[row] < 0
/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/3003789530.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.RSI_14MA = d

[[{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  36.774193548387096],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^NDX',
   'tp_sl_ratio': 2},
  37.06293706293706],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  41.17647058823529],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': '^SPX',
   'tp_sl_ratio': 2},
  33.88429752066116],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  43.53741496598639],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 2},
  38.732394366197184],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  45.88235294117647],
 [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 

In [289]:
# find thee param tha resulted in the highest score
# max(parameters_df, key=lambda x: x[1])

ticker_groups = {}
for item in parameters_df:
    ticker = item[0]['ticker']
    if ticker not in ticker_groups:
        ticker_groups[ticker] = []
    ticker_groups[ticker].append(item)

# Find the item with the highest value in index 1 for each ticker
result = {}
for ticker, items in ticker_groups.items():
    max_item = max(items, key=lambda x: x[1])
    result[ticker] = max_item

result

{'^NDX': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^NDX',
   'tp_sl_ratio': 1.5},
  45.88235294117647],
 '^SPX': [{'atr_length': 12,
   'init_size': 0.02,
   'slatr_coef': 9,
   'ticker': '^SPX',
   'tp_sl_ratio': 1.5},
  46.875],
 'AAPL': [{'atr_length': 1,
   'init_size': 0.02,
   'slatr_coef': 4,
   'ticker': 'AAPL',
   'tp_sl_ratio': 1.5},
  43.53741496598639]}

In [290]:
stat, bt = backtest("^SPX", atr_length=12, init_size=0.02 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/805540974.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2021-06-07 09:30:00
End                       2024-03-25 13:30:00
Duration                   1022 days 04:00:00
Exposure Time [%]                   83.736532
Equity Final [$]                383124.781298
Equity Peak [$]                 396626.663622
Return [%]                         283.124781
Buy & Hold Return [%]               23.857585
Return (Ann.) [%]                   61.626701
Volatility (Ann.) [%]               74.798532
Sharpe Ratio                         0.823903
Sortino Ratio                        2.169562
Calmar Ratio                         2.103778
Max. Drawdown [%]                  -29.293348
Avg. Drawdown [%]                   -3.422546
Max. Drawdown Duration      338 days 01:00:00
Avg. Drawdown Duration       10 days 12:00:00
# Trades                                   96
Win Rate [%]                           46.875
Best Trade [%]                       9.191412
Worst Trade [%]                      -3.24026
Avg. Trade [%]                    

In [305]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/2712909604.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2021-04-30 09:30:00
End                       2024-03-25 14:30:00
Duration                   1060 days 05:00:00
Exposure Time [%]                   83.798663
Equity Final [$]                787306.205566
Equity Peak [$]                  804194.38916
Return [%]                         687.306206
Buy & Hold Return [%]               31.297349
Return (Ann.) [%]                  104.069481
Volatility (Ann.) [%]              139.473377
Sharpe Ratio                          0.74616
Sortino Ratio                         2.65348
Calmar Ratio                         2.194274
Max. Drawdown [%]                  -47.427741
Avg. Drawdown [%]                   -3.874711
Max. Drawdown Duration      323 days 00:00:00
Avg. Drawdown Duration        8 days 15:00:00
# Trades                                   85
Win Rate [%]                        45.882353
Best Trade [%]                       21.26462
Worst Trade [%]                     -9.698482
Avg. Trade [%]                    

## Higheest return

In [248]:
stat2, bt2 = backtest("^NDX", atr_length=1, init_size=0.02 , tp_sl_ratio=2, slatr_coef=4)
bt.plot(show_legend=False)
stat2

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/41808923.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(show_legend=False)


Start                     2021-04-30 09:30:00
End                       2024-03-25 13:30:00
Duration                   1060 days 04:00:00
Exposure Time [%]                    69.79351
Equity Final [$]                535038.649414
Equity Peak [$]                 692238.485352
Return [%]                         435.038649
Buy & Hold Return [%]               31.331815
Return (Ann.) [%]                    78.56135
Volatility (Ann.) [%]                90.42001
Sharpe Ratio                         0.868849
Sortino Ratio                        2.772082
Calmar Ratio                         2.580232
Max. Drawdown [%]                  -30.447395
Avg. Drawdown [%]                    -2.86957
Max. Drawdown Duration      266 days 02:00:00
Avg. Drawdown Duration        8 days 12:00:00
# Trades                                  143
Win Rate [%]                        37.062937
Best Trade [%]                      13.674375
Worst Trade [%]                     -4.340308
Avg. Trade [%]                    

In [292]:
stat, bt = backtest("^NDX", atr_length=1, init_size=0.03 , tp_sl_ratio=1.5, slatr_coef=9)
bt.plot()
stat


/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_9219/519562990.py:2: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()


Start                     2021-04-30 09:30:00
End                       2024-03-25 14:30:00
Duration                   1060 days 05:00:00
Exposure Time [%]                   77.349587
Equity Final [$]               1701710.500977
Equity Peak [$]                1750078.258789
Return [%]                        1601.710501
Buy & Hold Return [%]               31.297349
Return (Ann.) [%]                  166.373702
Volatility (Ann.) [%]              244.905113
Sharpe Ratio                         0.679339
Sortino Ratio                        3.474715
Calmar Ratio                         3.553907
Max. Drawdown [%]                  -46.814307
Avg. Drawdown [%]                   -5.892652
Max. Drawdown Duration      262 days 00:00:00
Avg. Drawdown Duration        9 days 06:00:00
# Trades                                  105
Win Rate [%]                        38.095238
Best Trade [%]                       21.26462
Worst Trade [%]                      -5.53113
Avg. Trade [%]                    